In [ ]:
import os
#from dotenv import load_dotenv
from intake import open_catalog
import intake

import pandas as pd
import numpy as np
import geopandas as gpd

import datetime
from pystac_client import Client as psc
import stackstac
import numpy

#import dask.distributed


from pyproj import CRS
from pyproj import Transformer


In [ ]:
import rioxarray

In [ ]:
import rasterio.features
import xarray

In [ ]:
from shapely.geometry import mapping

In [ ]:
# Variables à changer pour récupérer les données sources de la chaîne des feux dans le système de l'oeil
# Utiliser intake et dotenv 
# Intake pour ouvrir le catalogue à partir de la db postgres de l'OEIL
# dotenv pour gérer les credentials et les chemins d'accès

PATH_GEOM = r"E:\FILES\OEIL\datas\sentinel_surfaces_detectees_sept_oct_2023.gpkg"

URL = "https://earth-search.aws.element84.com/v1"
URL_2 = "https://catalogue.dataspace.copernicus.eu/stac"
URL_3 = "https://services.sentinel-hub.com/api/v1/catalog/1.0.0/"
URL_4 = "https://earthengine.openeo.org/v1.0/"

collection = "SENTINEL-2"
collect_amazon ='sentinel-2-l2a'
collect_EE = 'COPERNICUS/S2_SR_HARMONIZED'

crs_rgnc = CRS.from_epsg(3163)
crs_4326 = CRS.from_epsg(4326)
transformer = Transformer.from_crs(crs_rgnc, crs_4326)

# On lit le fichier des surfaces brûlées dans un geodataframe
ba_test = gpd.read_file(PATH_GEOM)

# On créer une nouvelle colonne pour avoir un datetime de la date
ba_test['date_']= pd.to_datetime(ba_test['date'], format='%Y-%m-%d').dt.date

# On passe de multipolygon à polygon.
ba_test = ba_test.explode()
ba_test.head(2)

In [ ]:
ba_test[ba_test['surface_id']==359919]

In [ ]:
# On établit la liste des communes de notre geodataframe
liste_commune = list(set(ba_test["commune"]))

# On récupère les surfaces_id qui ont été photo interprétés, fournis par Oriane
surfaces_id = {
    "burned" : [358032, 358018, 358010, 359919, 359594, 359614, 358008, 359524, 359592, 359944],
    "unburned"  : [357997, 358001, 358002, 358017, 358012, 359543, 359788, 359498, 359545, 360203],
    "doubt": [358026, 358033,359595, 359964, 362134, 362171, 362192, 362851, 360718, 359666]
}

# On prend un intervalle de temps avec 120 jours avant la première date de détection de surface brûlée du geopackage des surfaces brûlées
datemin = (min(ba_test['date_']) - datetime.timedelta(days=120)).strftime('%Y-%m-%d') 
datemax = max(ba_test['date_']).strftime('%Y-%m-%d')

# On fait un test sur la surface brulée 358032
ba_test_filter = ba_test[ba_test["surface_id"]==358032]
# On créer une séries des dates de détection des surfaces brûlées pour filtrer les images sentinel2
dates_burnedarea = ba_test_filter['date_'].values
bbox = ba_test_filter["geometry"].to_crs(4326).total_bounds

# On prend un intervalle de temps avec 120 jours avant la première date de détection de surface brûlée du geopackage des surfaces brûlées
datemin = (min(ba_test_filter['date_']) - datetime.timedelta(days=120)).strftime('%Y-%m-%d') 
datemax = (max(ba_test_filter['date_'])+ datetime.timedelta(days=40)).strftime('%Y-%m-%d')

dates = f'{datemin}/{datemax}'
print(f'Emprise spatiale globale des formes identifiées {bbox}')

print(f'Interval temporel {dates}')


In [ ]:
client = psc.open(URL)
cc=40
search = client.search(
    collections=[collect_amazon],
    bbox=bbox,
    datetime=dates
    )

print(f"{search.matched()} scenes Sentinel-2 L2A trouvées dans l'intervalle temporel")


In [ ]:
items = search.item_collection()

In [ ]:
ba_test_filter

In [ ]:
sentinel_stack = stackstac.stack(
                          items,
                          bounds_latlon=[bbox[0], bbox[1],  bbox[2],  bbox[3]],

                          gdal_env=stackstac.DEFAULT_GDAL_ENV.updated(
                               {'GDAL_HTTP_MAX_RETRY': 3,
                                'GDAL_HTTP_RETRY_DELAY': 5,
                               }),
                          epsg=4326
                          ).rename({'x': 'lon', 'y': 'lat'})

In [ ]:
data_indices = sentinel_stack.sel(band=["blue","rededge2", "rededge3", "green","red", "nir","nir08","swir22", "scl"]).to_dataset(dim='band')
#rgb = sentinel_stack.sel(band=["red", "green", "blue"]).compute()


In [ ]:
for elt in data_indices.data_vars :
    print(type(elt))

In [ ]:
data_times = pd.to_datetime(sentinel_stack['time']).date
images_to_keep = []

for i, time in enumerate(data_times):
  if time in dates_burnedarea:
    images_to_keep.append(i)
    print(f"on conserve automatiquement l'image {i}")
    continue
  
  scl_data = sentinel_stack.isel(time = i).sel(band = "scl") 
  mask = (scl_data>=4) & (scl_data<=7)
  filtered_data = scl_data.where(mask)
  percentage = (filtered_data.count() / scl_data.count()) *100
  
  if percentage > 90:
    print(f"on prend l'image sufisamment peu couverte {i}")
    images_to_keep.append(i)

data_to_keep = sentinel_stack.isel(time=images_to_keep)

print(f"Nombre d'images après filtrage :{len(images_to_keep)}")

In [ ]:
data_indices = data_to_keep.sel(band=["blue","rededge2", "rededge3", "green","red", "nir","nir08","swir22", "scl"]).to_dataset(dim='band')
data_indices

In [ ]:
data_indices = data_to_keep.sel(band=["blue","rededge2", "rededge3", "green","red", "nir","nir08","swir22", "scl"]).to_dataset(dim='band')

"""
Correction des valeurs de bandes, on avait des valeurs négatives de reflectance par endroit
"""
"""
data_indices["red+"]=data_indices["red"].where(lambda x : x>0, lambda x : -x)
data_indices["nir+"]=data_indices["nir"].where(lambda x : x>0, lambda x : -x)
data_indices["swir22+"]=data_indices["swir22"].where(lambda x : x>0, lambda x : -x)
data_indices["green+"]=data_indices["green"].where(lambda x : x>0, lambda x : -x)
data_indices["blue+"]=data_indices["blue"].where(lambda x : x>0, lambda x : -x)
data_indices["nir08+"]=data_indices["nir08"].where(lambda x : x>0, lambda x : -x)
data_indices["rededge2+"]=data_indices["rededge2"].where(lambda x : x>0, lambda x : -x)
data_indices["rededge3+"]=data_indices["rededge3"].where(lambda x : x>0, lambda x : -x)
"""
data_indices["red"] = data_indices["red"]+0.1
data_indices["nir"] = data_indices["nir"]+0.1
data_indices["swir22"] = data_indices["swir22"]+0.1
data_indices["green"] = data_indices["green"]+0.1
data_indices["blue"] = data_indices["blue"]+0.1
data_indices["nir08"] = data_indices["nir08"]+0.1
data_indices["rededge2"] = data_indices["rededge2"]+0.1
data_indices["rededge3"] = data_indices["rededge3"]+0.1

In [ ]:
data_indices['ndvi'] = ((data_indices['nir'].astype(float) - data_indices['red'].astype(float))/(data_indices['nir'].astype(float)+ data_indices['red'].astype(float)))

data_indices['nbr'] = ((data_indices['nir'] - data_indices['swir22'])/(data_indices['nir'] + data_indices['swir22']))

data_indices['nbr+'] = ((data_indices['swir22'] - data_indices['nir08'] - data_indices['green'] - data_indices['blue'])/(data_indices['swir22'] + data_indices['nir08'] + data_indices['green'] + data_indices['blue']))

data_indices['bais2'] = (1-(numpy.sqrt((data_indices['rededge2'] * data_indices['rededge3'] * data_indices['nir08'])/data_indices['red']))*((data_indices['swir22'] - data_indices['nir08'] )/ numpy.sqrt((data_indices['swir22'] + data_indices['nir08'] ))+1))

In [ ]:
ShapeMask = rasterio.features.geometry_mask(ba_test_filter['geometry'].to_crs(4326).apply(mapping),
                                            out_shape =(len(data_indices.lat), len(data_indices.lon)),
                                            transform = data_indices.transform,
                                            invert = True)
ShapeMask = xarray.DataArray(ShapeMask, dims = ("lat","lon"))
data_indices['mask'] = ShapeMask
data_indices['mask'].plot()

In [ ]:
data_1D = data_indices.where(data_indices['mask']).data_vars['ndvi'].median(dim = ["lat", "lon"])
print(ba_test_filter["date_"].iloc[0])
data_1D.plot()

In [ ]:
data_indices["nbr+"].where(data_indices["mask"]).mean(dim=["lat", "lon"]).plot()

In [ ]:
data_indices["bais2"].where(data_indices["mask"]).mean(dim=["lat", "lon"]).plot()

In [ ]:
data_indices.time

In [ ]:
(data_indices['nbr'].isel(time=0)-data_indices['nbr'].isel(time=11)).plot(cmap = 'Reds')


In [ ]:
data_indices["nbr"].where(data_indices["mask"]).mean(dim=["lat", "lon"]).plot()

In [ ]:
data_indices["ndvi"].isel(time=12).plot.pcolormesh(cmap="YlGn")

In [ ]:
data_1D = data_indices.where(data_indices['mask']).data_vars['ndvi'].mean(dim = ["lat", "lon"])
print(ba_test_filter["date_"].iloc[0])
data_1D.plot()

In [ ]:
dataset = data_indices.to_dataarray(dim="bands")

In [ ]:
data_rgb = dataset.sel(bands=["red", "green", "blue"])

In [ ]:
ba_test_filter

In [ ]:
dataset_save = data_indices.drop([c for c in data_indices.coords if not (c in ['time', 'lat', 'lon'])])

In [ ]:
dataset_save = dataset_save.drop_vars([v for v in dataset_save.data_vars if not (v in ['ndvi', 'nbr', 'nbr+', 'bais2', 'mask'])])
dataset_save.attrs['spec'] = str(dataset_save.attrs['spec'])

In [ ]:
dataset_save.to_netcdf(f"{str(ba_test_filter["surface_id"].iloc[0])}_{str(ba_test_filter["nom"].iloc[0])}_{str(ba_test_filter["date"].iloc[0])}.nc")

In [ ]:
dataset_save

In [ ]:
dataset_save['ndvi'].where(dataset_save['mask']).mean(dim=["lat", 'lon']).plot()

In [ ]:
! pip install netCDF4